In [1]:
import pymrio as mr
import pandas as pd
import numpy as np
from scipy.linalg import block_diag
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

### Importer les données

In [2]:
EXIOBASE_PATH = '../EXIOBASE/' # ne pas hésiter à changer

In [3]:
year = 2019
filename = EXIOBASE_PATH+'IOT_{}_ixi.zip'.format(year)

In [4]:
io = mr.parse_exiobase3(path=filename)

In [5]:
population = pd.read_csv(EXIOBASE_PATH+'exiobase_population.txt',sep='\t',index_col=0)
io.population=pd.DataFrame(columns=['population'], data=population.loc[year].values, index=population.columns).T

In [6]:
io.calc_all()

C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:878: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  Y_agg = Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:928: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  F_Y_agg = self.F_Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:878: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  Y_agg = Y.sum(level="region", axis=1

### Agrégation des régions

Cette section consiste à agréger les régions entre elles. La fonctionnalité existe dans `pymrio`, il suffit d'assigner chaque région de la nomenclature existante à une nouvelle région pour préparer l'agrégation. La fonction `aggregate` s'occupe du reste.

In [18]:
io.get_regions()

Index(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR',
       'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO',
       'SE', 'SI', 'SK', 'GB', 'US', 'JP', 'CN', 'CA', 'KR', 'BR', 'IN', 'MX',
       'RU', 'AU', 'CH', 'TR', 'TW', 'NO', 'ID', 'ZA', 'WA', 'WL', 'WE', 'WF',
       'WM'],
      dtype='object', name='region')

In [8]:
new_regions = ['EUR'] * 28 + ['NAM'] + ['APAC'] + ['CN'] + ['NAM'] + ['APAC'] + ['LAM'] + ['IN'] + ['NAM'] + ['RU'] + ['APAC'] + ['EUR'] + ['EUR'] + ['APAC'] + ['EUR'] + ['APAC'] + ['AFR'] + ['APAC'] + ['LAM'] + ['EUR'] + ['AFR'] + ['MEA']

In [9]:
io_agg = io.aggregate(region_agg=new_regions, inplace=False)

C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:928: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  F_Y_agg = self.F_Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:960: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  self.D_cba.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:972: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  self.D_pba.sum(level="region", axis

In [10]:
io_agg.get_regions()

Index(['EUR', 'NAM', 'APAC', 'CN', 'LAM', 'IN', 'RU', 'AFR', 'MEA'], dtype='object', name='region')

In [11]:
io_agg.calc_all()

C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:878: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  Y_agg = Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:928: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  F_Y_agg = self.F_Y.sum(level="region", axis=1).reindex(


### Zoom sur un indicateur

La dernière version d'EXIOBASE contient les facteurs de caractérisation des gaz à effet de serre. Ces facteurs représentent le forçage radiatif cumulé généré par chaque gaz, sur 100 ans, comparé à celui du dioxyde de carbone. C'est le pouvoir de réchauffement global à 100 ans, en anglais GWP100.

La fonction `diag_stressor` permet de détailler l'origine des émissions pour un "stressor" (dans ce cas une catégorie d'impact) donné.

Celui qui nous intéresse est `GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)`.

In [22]:
list(io.impacts.get_index())

['Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
 'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
 'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
 'Damages to human health caused by climate change (H.A) | ECOINDICATOR 99 (H.A) | Damages to human health caused by climate change (H.A)',
 'Damage to Ecosystem Quality caused by ecotoxic emissions (H.A) | ECOINDICATOR 

In [23]:
# Créons l'extension correspondante
GHG = io_agg.impacts.diag_stressor('Water Withdrawal Blue - Total',
                             name = 'Eau')

In [24]:
# Connecter l'extension au système existant
io_agg.GHG = GHG

In [25]:
# Recalcul des impacts
io_agg.calc_all()

C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:878: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  Y_agg = Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:928: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  F_Y_agg = self.F_Y.sum(level="region", axis=1).reindex(
C:\Users\Gibon\.conda\envs\io\lib\site-packages\pymrio\core\mriosystem.py:960: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  self.D_cba.sum(level="region", axis=

In [26]:
# Vérifier que les calculs se sont bien déroulés
io_agg.meta

Description: Data for 2019
MRIO Name: exio381_ntnu_copy
System: ixi
Version: v3.81
File: None
History:
20211029 19:48:03 - MODIFICATION -  Calculating accounts for extension GHG
20211029 19:48:03 - MODIFICATION -  Calculating accounts for extension impacts
20211029 19:48:03 - MODIFICATION -  Calculating accounts for extension satellite
20211029 19:43:14 - MODIFICATION -  Calculating accounts for extension GHG
20211029 19:43:14 - MODIFICATION -  Calculating accounts for extension impacts
20211029 19:43:14 - MODIFICATION -  Calculating accounts for extension satellite
20211029 19:43:14 - MODIFICATION -  Calculating accounts for extension impacts
20211029 19:43:14 - MODIFICATION -  Calculating accounts for extension satellite
20211029 19:43:14 - MODIFICATION -  Leontief matrix L calculated
20211029 19:43:14 - MODIFICATION -  Coefficient matrix A calculated
 ... (more lines in history)

In [27]:
# Récupérer les flux de GES liés à la consommation finale
GHG_flows = io_agg.GHG.D_cba.groupby('region', axis=1).sum().groupby('region').sum()*1e-9

In [28]:
cmap  = 'viridis'

fig, ax = plt.subplots(ncols=2,
                       nrows=2,
                       figsize=(15, 10),
                       subplot_kw={'facecolor':'w'},
                       gridspec_kw={'width_ratios': [8, 1],
                                    'height_ratios': [8, 1],
                                    'wspace': 2/3*0.15,
                                    'hspace': 0.15,
                                    'bottom': 0})

sns.heatmap(GHG_flows, annot=True,
            norm=matplotlib.colors.LogNorm(vmin=.05,
                                           vmax=GHG_flows.max().max()),
            cbar=False,
            cmap=cmap,
           fmt='.0f',
           ax=ax[0,0])
sns.heatmap(pd.DataFrame(GHG_flows.sum(axis=1)), annot=True,
            norm=matplotlib.colors.LogNorm(vmin=.05,
                                           vmax=GHG_flows.max().max()),
            
            cmap=cmap,
           fmt='.0f',
           ax=ax[0,1])
sns.heatmap(pd.DataFrame(GHG_flows.sum()).T, annot=True,
#             annot_kws={'rotation': 90},
            norm=matplotlib.colors.LogNorm(vmin=.05,
                                           vmax=GHG_flows.max().max()),
            cbar=False,
            cmap=cmap,
           fmt='.0f',
           ax=ax[1,0])
sns.heatmap([[GHG_flows.sum().sum()]], annot=True,
            norm=matplotlib.colors.LogNorm(vmin=.05,
                                           vmax=GHG_flows.max().max()),
            cbar=False,
            cmap=cmap,
            yticklabels=[],
            xticklabels=['GLOBAL EMISSIONS'],
           fmt='.0f',
           ax=ax[1,1])

ax[0,0].set_xlabel('')
ax[1,1].set_xlabel('')
ax[1,0].set_xlabel('EMBODIED IN CONSUMPTION')
ax[1,0].set_ylabel('')
ax[1,0].set_yticklabels([])

# resize the corner box
ax01pos = ax[0,1].get_position().bounds
ax11pos = list(ax[1,1].get_position().bounds)
ax11pos[2] = ax01pos[2]
ax[1,1].set_position(ax11pos)

# fig.savefig('GHG_in_trade_{}.png'.format(year),
#             facecolor='white',
#             edgecolor='none')

ValueError: vmin must be less or equal to vmax

AttributeError: 'NoneType' object has no attribute 'reshape'

<Figure size 1080x720 with 4 Axes>